### Import Activities File

In [ ]:
import pandas as pd
from pathlib import Path
from os import listdir
from os.path import isfile, join
import gzip
import shutil
from benedict import benedict as bdict
from ipyleaflet import Map, Polyline


filepath = "data/"
fileOutputPath= filepath + "output/"

activityPath = filepath + "activities/"
activityOutputPath = fileOutputPath + "activities/"

def loadCleanData():
    df = pd.read_csv(filepath+ 'activities_clean.csv')
    df['Week'] = pd.to_datetime(df.Week)
    return df

def getFilename(fileextension):
    return fileextension[:-3]

def extractAllGPXFiles():
    # Create activity files Path if it's not exists
    Path(activityOutputPath).mkdir(parents=True, exist_ok=True)
    
    # check for all files in activities folder
    activityfiles = [f for f in listdir(filepath+"activities") if isfile(join(filepath+"activities", f))]

    # Extract all files from gz zip files
    for activityzip in activityfiles:
        # Extract activity filename
        filename=getFilename(activityzip)
       
        # Extract GPX File
        with gzip.open(activityPath + activityzip, 'rb') as f_in:
            with open(activityOutputPath + filename, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
    
    print("done!")
    
    
def getLocations(gpx_output_path,gpx_filename):
    # data-source can be an url, a filepath or data-string (as in this example)
    data_source = gpx_output_path + gpx_filename
    xmldata = bdict.from_xml(data_source)
    trk_list = xmldata['gpx.trk.trkseg.trkpt']
    
    locs = [[float(t['@lat']),float(t['@lon'])] for t in trk_list]
    
    return locs

def getVirtualLocations(gpx_output_path,gpx_filename):
    try:
        data_source = gpx_output_path + gpx_filename
        xmldata = bdict.from_xml(data_source)
        trk_list = xmldata['gpx.trk.trkseg.trkpt']

        locs = [[float(t['@lat']),float(t['@lon']),float(t['ele']),t['extensions']['power'],t['time']] for t in trk_list]
    
        return locs
    except:
        return null


def showOnMap(locations):

    line = Polyline(
        locations=locations,
        color="green" ,
        fill=False
    )

    m = Map(center = (float((min(locations)[0]+max(locations)[0])/2), float(min(locations)[1]+max(locations)[1])/2), zoom =12)
    m.add_layer(line)
    return m


In [ ]:
data = loadCleanData()

# Extract All gpx.gz files
#extractAllGPXFiles()

### General Stats

In [ ]:
# Total km by outdoor ride
outdoorRide = round(data[data['Activity Type']=='Ride']['Distance'].sum())
print("Total outdoor ride: ", outdoorRide)

# Total km by outdoor ride
virtualRide = round(data[data['Activity Type']=='Virtual Ride']['Distance'].sum())
print("Total virtual ride: ", virtualRide)

In [ ]:
# Ride Distance Graph
import seaborn as sns
sns.set()
sns.barplot(x="Activity Type", y="Distance", data=data, estimator=sum)

In [ ]:
# Ride Elevation Graph
sns.barplot(x="Activity Type", y="Elevation Gain", data=data, estimator=sum)

In [ ]:
import matplotlib.pyplot as plt

sns.distplot(data['Distance'], kde=False)
plt.axvline(0, color="k", linestyle="--")

In [ ]:
# Distance Graph per day
distGraph = sns.lineplot(data=data, x="Activity Date", y="Distance", hue="Activity Type")
distGraph.set(xticklabels=[]) 
distGraph.set(xlabel=None)
distGraph.tick_params(bottom=False) 
distGraph

### Data Visualization

In [ ]:
from matplotlib import pyplot as plt  

In [ ]:
x = list(data['Activity Date'])
y = list(data['Elevation Gain'])
  
plt.plot(x,y) 
plt.show()  

In [ ]:
import seaborn as sns
sns.set()

In [ ]:
plt.plot(y)
plt.legend('Elevation Gain', ncol=2, loc='upper left');

### Data Visualization with Seaborn

In [ ]:
with sns.axes_style('white'):
    sns.jointplot("Distance", "Elevation Gain", data, kind='kde');

In [ ]:
with sns.axes_style('white'):
    sns.jointplot("Distance", "Elevation Gain", data, kind='hex')

In [ ]:
sns.jointplot("Distance", "Elevation Gain", data=data, kind='reg');

In [ ]:
g = sns.PairGrid(data, vars=['Distance', 'Max Speed', 'Elevation Gain', 'Average Watts'],
                 hue='Activity Type', palette='RdBu_r')
g.map(plt.scatter, alpha=0.8)
g.add_legend();

### Graph Time Distribution

In [ ]:
# One sample for a Power per ride
len(getVirtualLocations(fileOutputPath,getFilename(data['Filename'][155])))

In [ ]:
virtualSteps = getVirtualLocations(fileOutputPath,getFilename(data['Filename'][155]))

import pandas as pd 
  
dfSteps = pd.DataFrame(virtualSteps, columns =['Latitude', 'Longtitude','Elevation','Power','Date']) 
dfSteps = dfSteps.astype({"Power": int})

powerPlot = sns.lineplot(data=dfSteps, x="Date", y="Power")
powerPlot.set(xticklabels=[]) 
powerPlot.set(xlabel=None)
powerPlot.tick_params(bottom=False) 
powerPlot

### Modify Data to Generate New Features

In [ ]:
data['Time/Distance'] = data['Distance.1'] / data['Elapsed Time.1']
data.head(5)

### Data Visualization Per Column

In [ ]:
data.plot(kind='scatter',x='index',y='Activity Type')

In [ ]:
data.plot(kind='scatter',x='index',y='Distance')

In [ ]:
data.plot(kind='scatter',x='index',y='Elapsed Time')

In [ ]:
data.plot(kind='scatter',x='index',y='Moving Time')

In [ ]:
data.plot(kind='scatter',x='index',y='Max Speed')

In [ ]:
data.plot(kind='scatter',x='index',y='Elevation Gain')

In [ ]:
data.plot(kind='scatter',x='index',y='Calories')

In [ ]:
data.plot(kind='scatter',x='index',y='Activity Name')

In [ ]:
data.plot(kind='scatter',x='index',y='Average Watts')

In [ ]:
data.plot(kind='scatter',x='index',y='Average Cadence')